In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="qftentangled",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------",qc.name, "with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

--------------- qftentangled with qubit 3 ---------------
--------------- qftentangled with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x72d39f18e1e0>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x72d39f13b440>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'010': 109, '000': 272, '011': 32, '111': 224, '101': 38, '001': 205, '110': 120}, {'000': 0.24999999999999994, '001': 0.21338834764831827, '010': 0.12499999999999978, '011': 0.0366116523516815, '100': 3.1006617162384195e-32, '101': 0.03661165235168155, '110': 0.12499999999999994, '111': 0.2133883476483186})
(4, {'1101': 91, '0110': 19, '0100': 62, '0001': 103, '1111': 116, '1001': 1, '0101': 45, '1110': 100, '1100': 65, '0010': 104, '0000': 129, '0111': 3, '0011': 91, '1011': 48, '1010': 23}, {'0000': 0.12499999999999982, '0001': 0.12024247078195532, '0010': 0.10669417382415898, '0011': 0.0864177145228181, '0100': 0.06249999999999986, '0101': 0.038582285477181885, '0110': 0.01830582617584072, '0111': 0.00475752921804458, '1000': 9.451750926374887e-32, '1001': 0.00475752921804458, '1010': 0.01830582617584078, '1011': 0.03858228547718189, '1100': 0.06249999999999992, '1101': 0.08641771452281813, '1110': 0.10669417382415912, '1111': 0.1202424707819555})


In [3]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [4]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (qft, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (qft, 3 qubits): 0.9986
Hellinger Fidelity (qft, 4 qubits): 0.9964
